In [12]:
"""for pulling down data from hr.info API on response level and sending to CSV"""

import urllib.request, json
import csv
import itertools
import math
from collections import OrderedDict

import grequests
import pandas as pd
from pandas.io.json import json_normalize

In [10]:
#pulling data

def pull_indiv_rw(data):
    """
    return all individual crisis data, exclude ones that don't have a 200 status code.
    
    pull into a df and return, then that gets merged with the existing df
    """
    hrefz = data['rw_gen.href']

    def exception_handler(request, exception):
         print('Bad URL for ' + str(request))

    resps = []
    it = 200
    for v in range(0, len(hrefz), it):
        print('Pulling individual for hrefs to ' + str(v))
        rs = (grequests.get(ref) for ref in hrefz[v : v+it])
        resps += grequests.map(rs, exception_handler = exception_handler)
    
    resps = [json.loads(r.content) for r in resps if r.status_code == 200]
        
    return json_normalize(resps)[['data', 'href', 'totalCount']].add_prefix('rw_gen.')
#     return json_normalize(resps, ['totalCount', 'data', 'href']).add_prefix('rw_gen.')

def fetch_api_rw(maxv = None):
    """
    pull down all API info for RW general crisis and return as dataframe.
    
    maxv = (start_val, end_val)
    """
    data = []    
    
    if maxv:
        start = maxv[0]
        fin = maxv[1]
        step = min(fin - start, 1000)
    
    else:
        start = 0
        fin = json.loads(urllib.request.urlopen("https://api.reliefweb.int/v1/" \
                                   "disasters?appname=vocabulary&preset=external").read().decode())['totalCount']
        step = 1000
        
    for i in range(start, fin, step):
        with urllib.request.urlopen("https://api.reliefweb.int/v1/disasters?appname=vocabulary"
                                    "&preset=external&limit={0}&offset={1}"
                                    .format(min(step, fin - i), i)) as url:

            data += json.loads(url.read().decode())['data']


    ret = json_normalize(data).add_prefix('rw_gen.')
    
    assert(len(ret) == fin - start)
    print('pulling down from rw entry count of: ' + str(len(ret)))
    return ret

In [1]:
nd = rw(test = (1450,1680), year = 2005)
r = nd.master_pull()

l = pull_indiv_rw(r)
j = json_normalize(l['rw_gen.data'][0])


for p in l:
    print(json_normalize(p['rw_gen.data'][0]))
#     print(json_normalize(l['rw_gen.data'][0])['fields.country'][0])
#     for v in json_normalize(l['rw_gen.data'][0]):
#         print(v)
#         if 'primary' in v:
#             primary = v
#             break

print(j['fields.country'][0])
        
# [v for v in j.iteritems()]

j

NameError: name 'rw' is not defined

In [246]:
def get_spec_crisis(ent):
    """
    merge relevant crisis level data
    """
    global ret
    j = json_normalize(ent['rw_gen.data'])
    primary = None

    for v in j['fields.country'][0]:
        if 'primary' in v:
            primary = v
            break

    #bad news if no primary
    if not primary:
        print('*** no primary! ' + str(v))
        ret = [None] * (len(self.all_cols) - len(self.new_cols_top))


    as_int = lambda x : print(j[x])
    as_is = lambda x : j[x][0] if x in j.columns else None
    len_country = lambda x : len(j['fields.country']) if 'fields.country' in j else None
    from_prim = lambda x : primary[x] if x in primary else None
    def from_prim_loc(col):
        ret = None
        if col == 'fields.country.location_lat':
            if 'location' in primary:
                return primary['location']['lat']
        elif col == 'fields.country.location_long':
            if 'location' in primary:
                return primary['location']['lon']



    p = lambda x : None

    get_vals = OrderedDict({
    #         'rw_gen.totalCount' : as_int,
            'fields.description' : as_is,
            'fields.url_alias' : as_is,
            'fields.primary_type.id' : as_is,
            'fields.primary_type.name' : as_is,
            'fields.primary_type.code' : as_is,
            'fields.country.name' : from_prim,
            'fields.country.iso' : from_prim,
            'fields.country.href' : from_prim,
            'fields.country.location_lat' : from_prim_loc,
            'fields.country.location_long' : from_prim_loc,
            'num_country' : len_country})

    ret = [{x, get_vals[x](x)} for x in get_vals.keys()] 
    
    
#     df = pd.DataFrame(columns = list(get_vals.keys()))
#     df.append(ret)



# get_spec_crisis()
#         l[list(get_vals.keys())] = l.apply(lambda x : pd.Series(self._get_spec_crisis_lamb(x)), axis = 1)

        #drop unnecessary columns, cleanup
#         l = l.drop('rw_gen.data', axis = 1)
#         l = l.drop('rw_gen.totalCount', axis = 1)
#         l['num_country'] = l['num_country'].astype(int)

#         self.data = self.data.merge(l, how = 'left', on = 'rw_gen.href')

https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=30&offset=1450
pulling down from rw entry count of: 30
trim data length: 30
Pulling individual for hrefs to 0
Pulling individual for hrefs to 0


AttributeError: 'list' object has no attribute 'values'

In [ ]:
def _get_spec_crisis_lamb(self, v):
        global ret
        global j

        ret = []

        if v['rw_gen.totalCount'] != 1:
            print('***wrong totalCount ' + str(v))

        j = json_normalize(v['rw_gen.data'][0])

        #add in top level data compents
        try:
            ret += [j[ent.split('data.')[1]][0] for ent in self.new_cols_top]
            
        except:
            ret += [None] * len(self.new_cols_top)

        #add in data.fields info. entry be like:
        """
            {'href': 'https://api.reliefweb.int/v1/countries/255',
             'id': 255,
             'iso3': 'yem',
             'location': {'lat': 15.94, 'lon': 47.62},
             'name': 'Yemen',
             'primary': True}
            primary = None
        """
        
        #add in other columns
        for v in j['fields.country'][0]:
            if 'primary' in v:
                primary = v
                break

        #bad news if no primary
        if not primary:
            print('*** no primary! ' + str(v))
            ret = [None] * (len(self.all_cols) - len(self.new_cols_top))
            
        else:
            try:
                rollback = ret
                
                #also add in other cols
                ret.append(len(j['fields.country'][0]))

                ret += [primary[c] for c in ['name', 'iso3', 'href']]
                ret.append(primary['location']['lat'])
                ret.append(primary['location']['lon'])
                
            except:
                ret = rollback + [None] * (len(rollback) - (len(self.all_cols) - len(self.new_cols_top)))
                
        return ret
    
    def get_spec_crisis(self):
        """
        merge relevant crisis level data
        """
        
        l = pull_indiv_rw(self.data)
        
        l['rw_gen.totalCount'] = l['rw_gen.totalCount'].astype(int)

        self.new_cols_top = 'data.fields.description',
                            'data.fields.url_alias',
                            'data.primary_type.id',
                            'data.primary_type.name',
                            'data.primary_type.code'
                            ]
    
        self.other_cols = ['num_country']
        
        self.new_cols_country = 'data.fields.country.name', 
                                'data.fields.country.iso', 
                                'data.fields.country.href',
                                'data.fields.country.location_lat', 
                                'data.fields.country.location_long']

        self.all_cols = self.new_cols_top + self.other_cols + self.new_cols_country

        for v in self.all_cols:
            l[v] = None
        
        l[self.all_cols] = l.apply(lambda x : pd.Series(self._get_spec_crisis_lamb(x)), axis = 1)

        #drop unnecessary columns, cleanup
        l = l.drop('rw_gen.data', axis = 1)
        l = l.drop('rw_gen.totalCount', axis = 1)
        l['num_country'] = l['num_country'].astype(int)

        self.data = self.data.merge(l, how = 'left', on = 'rw_gen.href')

In [8]:
sc_dat = pull()

loaded 227 SC entries


In [43]:
ret

,rw_gen.data,rw_gen.href,rw_gen.totalCount,rw.dis_type
0,"[{'fields': {'id': 6291, 'name': 'Solomon Isla...",https://api.reliefweb.int/v1/disasters/6291,1,NaN
1,"[{'fields': {'id': 6190, 'name': 'Argentina: F...",https://api.reliefweb.int/v1/disasters/6190,1,NaN
2,"[{'fields': {'id': 5355, 'name': 'OPT: Waste W...",https://api.reliefweb.int/v1/disasters/5355,1,NaN
3,"[{'fields': {'id': 5150, 'name': 'Haiti: Flood...",https://api.reliefweb.int/v1/disasters/5150,1,NaN
4,"[{'fields': {'id': 5758, 'name': 'South Asia: ...",https://api.reliefweb.int/v1/disasters/5758,1,NaN
5,"[{'fields': {'id': 6646, 'name': 'Afghanistan:...",https://api.reliefweb.int/v1/disasters/6646,1,NaN
6,"[{'fields': {'id': 5936, 'name': 'Madagascar: ...",https://api.reliefweb.int/v1/disasters/5936,1,NaN
7,"[{'fields': {'id': 5211, 'name': 'Indonesia: E...",https://api.reliefweb.int/v1/disasters/5211,1,NaN
8,"[{'fields': {'id': 5201, 'name': 'Indonesia: L...",https://api.reliefweb.int/v1/disasters/5201,1,NaN
9,"[{'fields': {'id': 6248, 'name': 'Paraguay: De...",https://api.reliefweb.int/v1/disasters/6248,1,NaN


In [81]:
ret.apply(lambda x : x['rw_gen.data'][0]['fields']['primary_type']['name'], axis = 1)

0                   Tsunami
1                     Flood
2    Technological Disaster
3                     Flood
4                Land Slide
5                     Flood
6          Tropical Cyclone
7                Earthquake
8                Land Slide
9                  Epidemic
dtype: object

In [75]:
ret['rw_gen.data'][0][0]['fields']['primary_type']['name']

'Tsunami'

In [91]:
r

,rw_gen.fields.glide,rw_gen.fields.id,rw_gen.fields.name,rw_gen.fields.status,rw_gen.href,rw_gen.id,rw_gen.score,rw_gen.month,rw_gen.year,data.fields.description,data.fields.url_alias,num_country,data.fields.country.name,data.fields.country.iso,data.fields.country.href,data.fields.country.location_lat,data.fields.country.location_long,rw.uid,rw.in_sc
0,TS-2007-000042-SLB,6291,Solomon Islands: Earthquake and Tsunami,past,https://api.reliefweb.int/v1/disasters/6291,6291,1,Apr,2007,None,None,1,Solomon Islands,slb,https://api.reliefweb.int/v1/countries/215,-8.92,159.63,SLB2007,False
1,FL-2007-000044-ARG,6190,Argentina: Floods,past,https://api.reliefweb.int/v1/disasters/6190,6190,1,Mar,2007,None,None,1,Argentina,arg,https://api.reliefweb.int/v1/countries/22,-35.38,-64.00,ARG2007,False
2,AC-2007-000041-PSE,5355,OPT: Waste Water Treatment Plant Floods,past,https://api.reliefweb.int/v1/disasters/5355,5355,1,Mar,2007,None,None,2,Israel,isr,https://api.reliefweb.int/v1/countries/125,31.09,34.87,ISR2007,False
3,FL-2007-000040-HTI,5150,Haiti: Floods,past,https://api.reliefweb.int/v1/disasters/5150,5150,1,Mar,2007,None,None,1,Haiti,hti,https://api.reliefweb.int/v1/countries/113,19.18,-72.43,HTI2007,False
4,LS-2007-000037-PAK,5758,South Asia: Floods and landslides,past,https://api.reliefweb.int/v1/disasters/5758,5758,1,Mar,2007,None,None,2,India,ind,https://api.reliefweb.int/v1/countries/119,22.76,79.28,IND2007,False
5,FL-2007-000035-AFG,6646,Afghanistan: Floods and avalanche,past,https://api.reliefweb.int/v1/disasters/6646,6646,1,Mar,2007,None,None,1,Afghanistan,afg,https://api.reliefweb.int/v1/countries/13,33.84,66.03,AFG2007,False
6,TC-2007-000034-MDG,5936,Madagascar: Cyclone Indlala,past,https://api.reliefweb.int/v1/disasters/5936,5936,1,Mar,2007,Cyclone Indlala hit the north-eastern coast of...,https://reliefweb.int/disaster/tc-2007-000034-mdg,1,Madagascar,mdg,https://api.reliefweb.int/v1/countries/144,-19.37,46.71,MDG2007,False
7,EQ-2007-000033-IDN,5211,Indonesia: Earthquake,past,https://api.reliefweb.int/v1/disasters/5211,5211,1,Mar,2007,None,None,1,Indonesia,idn,https://api.reliefweb.int/v1/countries/120,-2.28,117.37,IDN2007,False
8,LS-2007-000032-IDN,5201,Indonesia: Landslides,past,https://api.reliefweb.int/v1/disasters/5201,5201,1,Mar,2007,None,None,1,Indonesia,idn,https://api.reliefweb.int/v1/countries/120,-2.28,117.37,IDN2007,False
9,EP-2007-000031-PRY,6248,Paraguay: Dengue Outbreak,past,https://api.reliefweb.int/v1/disasters/6248,6248,1,Feb,2007,None,None,1,Paraguay,pry,https://api.reliefweb.int/v1/countries/186,-23.24,-58.39,PRY2007,False


In [ ]:
data

In [103]:
ret

,rw_gen.fields.glide,rw_gen.fields.id,rw_gen.fields.name,rw_gen.fields.status,rw_gen.href,rw_gen.id,rw_gen.score,rw_gen.month,rw_gen.year,data.fields.description,data.fields.url_alias,num_country,data.fields.country.name,data.fields.country.iso,data.fields.country.href,data.fields.country.location_lat,data.fields.country.location_long,rw.uid,rw.in_sc,rw.disaster_type
0,TS-2007-000042-SLB,6291,Solomon Islands: Earthquake and Tsunami,past,https://api.reliefweb.int/v1/disasters/6291,6291,1,Apr,2007,None,None,1,Solomon Islands,slb,https://api.reliefweb.int/v1/countries/215,-8.92,159.63,SLB2007,False,Tsunami
1,FL-2007-000044-ARG,6190,Argentina: Floods,past,https://api.reliefweb.int/v1/disasters/6190,6190,1,Mar,2007,None,None,1,Argentina,arg,https://api.reliefweb.int/v1/countries/22,-35.38,-64.00,ARG2007,False,Flood
2,AC-2007-000041-PSE,5355,OPT: Waste Water Treatment Plant Floods,past,https://api.reliefweb.int/v1/disasters/5355,5355,1,Mar,2007,None,None,2,Israel,isr,https://api.reliefweb.int/v1/countries/125,31.09,34.87,ISR2007,False,Technological Disaster
3,FL-2007-000040-HTI,5150,Haiti: Floods,past,https://api.reliefweb.int/v1/disasters/5150,5150,1,Mar,2007,None,None,1,Haiti,hti,https://api.reliefweb.int/v1/countries/113,19.18,-72.43,HTI2007,False,Flood
4,LS-2007-000037-PAK,5758,South Asia: Floods and landslides,past,https://api.reliefweb.int/v1/disasters/5758,5758,1,Mar,2007,None,None,2,India,ind,https://api.reliefweb.int/v1/countries/119,22.76,79.28,IND2007,False,Land Slide
5,FL-2007-000035-AFG,6646,Afghanistan: Floods and avalanche,past,https://api.reliefweb.int/v1/disasters/6646,6646,1,Mar,2007,None,None,1,Afghanistan,afg,https://api.reliefweb.int/v1/countries/13,33.84,66.03,AFG2007,False,Flood
6,TC-2007-000034-MDG,5936,Madagascar: Cyclone Indlala,past,https://api.reliefweb.int/v1/disasters/5936,5936,1,Mar,2007,Cyclone Indlala hit the north-eastern coast of...,https://reliefweb.int/disaster/tc-2007-000034-mdg,1,Madagascar,mdg,https://api.reliefweb.int/v1/countries/144,-19.37,46.71,MDG2007,False,Tropical Cyclone
7,EQ-2007-000033-IDN,5211,Indonesia: Earthquake,past,https://api.reliefweb.int/v1/disasters/5211,5211,1,Mar,2007,None,None,1,Indonesia,idn,https://api.reliefweb.int/v1/countries/120,-2.28,117.37,IDN2007,False,Earthquake
8,LS-2007-000032-IDN,5201,Indonesia: Landslides,past,https://api.reliefweb.int/v1/disasters/5201,5201,1,Mar,2007,None,None,1,Indonesia,idn,https://api.reliefweb.int/v1/countries/120,-2.28,117.37,IDN2007,False,Land Slide
9,EP-2007-000031-PRY,6248,Paraguay: Dengue Outbreak,past,https://api.reliefweb.int/v1/disasters/6248,6248,1,Feb,2007,None,None,1,Paraguay,pry,https://api.reliefweb.int/v1/countries/186,-23.24,-58.39,PRY2007,False,Epidemic


In [106]:
r

,rw_gen.fields.glide,rw_gen.fields.id,rw_gen.fields.name,rw_gen.fields.status,rw_gen.href,rw_gen.id,rw_gen.score,rw_gen.month,rw_gen.year,data.fields.description,data.fields.url_alias,num_country,data.fields.country.name,data.fields.country.iso,data.fields.country.href,data.fields.country.location_lat,data.fields.country.location_long,rw.uid,rw.in_sc,rw.disaster_type
0,TS-2007-000042-SLB,6291,Solomon Islands: Earthquake and Tsunami,past,https://api.reliefweb.int/v1/disasters/6291,6291,1,Apr,2007,None,None,1,Solomon Islands,slb,https://api.reliefweb.int/v1/countries/215,-8.92,159.63,SLB2007,False,Tsunami
1,FL-2007-000044-ARG,6190,Argentina: Floods,past,https://api.reliefweb.int/v1/disasters/6190,6190,1,Mar,2007,None,None,1,Argentina,arg,https://api.reliefweb.int/v1/countries/22,-35.38,-64.00,ARG2007,False,Flood
2,AC-2007-000041-PSE,5355,OPT: Waste Water Treatment Plant Floods,past,https://api.reliefweb.int/v1/disasters/5355,5355,1,Mar,2007,None,None,2,Israel,isr,https://api.reliefweb.int/v1/countries/125,31.09,34.87,ISR2007,False,Technological Disaster
3,FL-2007-000040-HTI,5150,Haiti: Floods,past,https://api.reliefweb.int/v1/disasters/5150,5150,1,Mar,2007,None,None,1,Haiti,hti,https://api.reliefweb.int/v1/countries/113,19.18,-72.43,HTI2007,False,Flood
4,LS-2007-000037-PAK,5758,South Asia: Floods and landslides,past,https://api.reliefweb.int/v1/disasters/5758,5758,1,Mar,2007,None,None,2,India,ind,https://api.reliefweb.int/v1/countries/119,22.76,79.28,IND2007,False,Land Slide
5,FL-2007-000035-AFG,6646,Afghanistan: Floods and avalanche,past,https://api.reliefweb.int/v1/disasters/6646,6646,1,Mar,2007,None,None,1,Afghanistan,afg,https://api.reliefweb.int/v1/countries/13,33.84,66.03,AFG2007,False,Flood
6,TC-2007-000034-MDG,5936,Madagascar: Cyclone Indlala,past,https://api.reliefweb.int/v1/disasters/5936,5936,1,Mar,2007,Cyclone Indlala hit the north-eastern coast of...,https://reliefweb.int/disaster/tc-2007-000034-mdg,1,Madagascar,mdg,https://api.reliefweb.int/v1/countries/144,-19.37,46.71,MDG2007,False,Tropical Cyclone
7,EQ-2007-000033-IDN,5211,Indonesia: Earthquake,past,https://api.reliefweb.int/v1/disasters/5211,5211,1,Mar,2007,None,None,1,Indonesia,idn,https://api.reliefweb.int/v1/countries/120,-2.28,117.37,IDN2007,False,Earthquake
8,LS-2007-000032-IDN,5201,Indonesia: Landslides,past,https://api.reliefweb.int/v1/disasters/5201,5201,1,Mar,2007,None,None,1,Indonesia,idn,https://api.reliefweb.int/v1/countries/120,-2.28,117.37,IDN2007,False,Land Slide
9,EP-2007-000031-PRY,6248,Paraguay: Dengue Outbreak,past,https://api.reliefweb.int/v1/disasters/6248,6248,1,Feb,2007,None,None,1,Paraguay,pry,https://api.reliefweb.int/v1/countries/186,-23.24,-58.39,PRY2007,False,Epidemic


In [105]:
from sc_pull import pull
nd = rw(test = (1450,1460), year = 2005, sc = sc_dat)
r = nd.master_pull()

l = pull_indiv_rw(r)
j = json_normalize(l['rw_gen.data'][0])


pulling down from rw entry count of: 10
trim data length: 10
Pulling individual for hrefs to 0
Pulling individual disaster types for hrefs to 0
Pulling individual for hrefs to 0


In [104]:
class rw(object):
    def t(self):
        pass

    def __init__(self, year, sc, test = None):
        self.test = test
        self.data = None
        self.year = year
        self.sc = sc

    def add_crisis_type(self):
        """
        patchy patch for pulling crisis type from RW and adding to existing data
        """
        hrefz = self.data['rw_gen.href']

        def exception_handler(request, exception):
             print('Bad URL for ' + str(request))

        resps = []
        it = 200
        for v in range(0, len(hrefz), it):
            print('Pulling individual disaster types for hrefs to ' + str(v))
            rs = (grequests.get(ref) for ref in hrefz[v : v+it])
            resps += grequests.map(rs, exception_handler = exception_handler)

        resps = [json.loads(r.content) for r in resps if r.status_code == 200]
        info =  json_normalize(resps)[['data', 'href', 'totalCount']]
        
        info['rw.disaster_type'] = info.apply(lambda x : x['data'][0]['fields']['primary_type']['name'], axis = 1)
        self.data = data.merge(info[['rw.disaster_type', 'href']], left_on = 'rw_gen.href', right_on = 'href')
        self.data.drop('href', axis = 1, inplace=True)

    def extract_date(self, val):
        name = val.replace(' ', '')[-7:]
        month = None
        year = None

        MIN_YEAR = 2005
        # if we don't have regular formatting, take just year
        if not name[0:3].isalpha():
            if not name[-4:].isnumeric():
                print("***bad year: " + name)
            else:
                year = int(name[-4:])
        else:
            month = name[:3]
            year = int(name[-4:])

        if year:
            if year >= MIN_YEAR:
                return [month, year]

        return (None, None)

    def get_date(self):
        self.data['rw_gen.month'] = None
        self.data['rw_gen.year'] = None
        self.data[['rw_gen.month', 'rw_gen.year']] = self.data.apply(lambda x:
                                                                     pd.Series(
                                                                         self.extract_date(x['rw_gen.fields.name'])),
                                                                     axis=1)

        self.data = self.data[pd.notnull(self.data['rw_gen.year'])]
        self.data['rw_gen.year'] = self.data['rw_gen.year'].astype(int)

    def trim_nm(self):

        def trim(v):
            s_val = None

            if len(v.split('-')) != 1:
                s_val = '-'
            elif len(v.split('–')) != 1:
                s_val = '–'

            if s_val:
                return v.split(s_val)[0]
            else:
                return v

        self.data['rw_gen.fields.name'] = self.data.apply(lambda x: trim(x['rw_gen.fields.name']), axis=1)

    def rm_old(self):
        self.data = self.data.drop(self.data[self.data['rw_gen.year'] < self.year].index)
        print('trim data length: ' + str(len(self.data)))

    def _get_spec_crisis_lamb(self, v):
        ret = []

        if v['rw_gen.totalCount'] != 1:
            print('***wrong totalCount ' + str(v))

        j = json_normalize(v['rw_gen.data'][0])

        # add in top level data compents
        try:
            ret += [j[ent.split('data.')[1]][0] for ent in self.new_cols_top]

        except:
            ret += [None] * len(self.new_cols_top)

        # add in data.fields info. entry be like:
        """
            {'href': 'https://api.reliefweb.int/v1/countries/255',
             'id': 255,
             'iso3': 'yem',
             'location': {'lat': 15.94, 'lon': 47.62},
             'name': 'Yemen',
             'primary': True}
            primary = None
        """

        # add in other columns
        for v in j['fields.country'][0]:
            if 'primary' in v:
                primary = v
                break

                # bad news if no primary
        if not primary:
            print('*** no primary! ' + str(v))
            ret = [None] * (len(self.all_cols) - len(self.new_cols_top))

        else:
            try:
                rollback = ret

                # also add in other cols
                ret.append(len(j['fields.country'][0]))

                ret += [primary[c] for c in ['name', 'iso3', 'href']]
                ret.append(primary['location']['lat'])
                ret.append(primary['location']['lon'])

            except:
                ret = rollback + [None] * (len(rollback) - (len(self.all_cols) - len(self.new_cols_top)))

        return ret

    def get_spec_crisis(self):
        """
        merge relevant crisis level data
        """

        l = pull_indiv_rw(self.data)

        l['rw_gen.totalCount'] = l['rw_gen.totalCount'].astype(int)

        self.new_cols_top = ['data.fields.description',
                             'data.fields.url_alias']

        self.other_cols = ['num_country']

        self.new_cols_country = ['data.fields.country.name',
                                 'data.fields.country.iso',
                                 'data.fields.country.href',
                                 'data.fields.country.location_lat',
                                 'data.fields.country.location_long']

        self.all_cols = self.new_cols_top + self.other_cols + self.new_cols_country

        for v in self.all_cols:
            l[v] = None

        l[self.all_cols] = l.apply(lambda x: pd.Series(self._get_spec_crisis_lamb(x)), axis=1)

        # drop unnecessary columns, cleanup
        l = l.drop('rw_gen.data', axis=1)
        l = l.drop('rw_gen.totalCount', axis=1)
        l['num_country'] = l['num_country'].astype(int)

        self.data = self.data.merge(l, how='left', on='rw_gen.href')

    def add_uid(self):
        self.data['rw.uid'] = self.data['data.fields.country.iso'].str.upper() + self.data['rw_gen.year'].map(str)

    def check_in_sc(self):
        self.data['rw.in_sc'] = self.data['rw.uid'].isin(self.sc['sc.uid'])

    def master_pull(self):
        """take crises only after certain year, add month_crisis: mmm, and year_crisis: yyyy to each crisis's entry

            names are either in format of:
                MMM YYYY
                OR
                YYYY-YYYY

            if not in first format, check to see if end year > 2005
        """
        # only pull some data if test
        if self.test:
            self.data = fetch_api_rw(self.test)
        else:
            self.data = fetch_api_rw()

        # do things to primary rw data
        self.get_date()
        self.rm_old()
        self.trim_nm()

        # do crisiswise pull
        self.get_spec_crisis()
        self.add_crisis_type()

        #sc, uid
        self.add_uid()
        self.check_in_sc()

        return self.data